<a href="https://colab.research.google.com/github/Akhil-ender/Fine-Tuning-LLM-s-for-Classification/blob/main/Flan_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import sys
# Mount the Google Drive to access files stored there
drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/NLP/Class_notebooks/Custom_Funtions')

# Install the latest version of torchtext library quietly without showing output
!pip install torchtext -qq
!pip install transformers evaluate wandb datasets accelerate -U -qq ## NEW LINES ##
basepath = '/content/drive/MyDrive/data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# standard data science librraies for data handling and v isualization
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import CustomPreprocessorSpacy as cp
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, f1_score



import evaluate
from datasets import Dataset
from torch.utils.data import  DataLoader, RandomSampler, SequentialSampler
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer
from transformers import T5ForSequenceClassification
from transformers import T5Config
from transformers import pipeline
from transformers import EarlyStoppingCallback
from transformers import pipeline
from datasets import load_dataset, load_metric
import numpy as np

import wandb

In [ ]:
model_folder = '/content/drive/MyDrive/NLP/Assignments/HW6/Flan_T5'

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/NLP/Assignments/HW6/train.csv')
test = pd.read_csv('/content/drive/MyDrive/NLP/Assignments/HW6/test.csv')

In [ ]:
X_train = train['Tweet']
X_test = test['Tweet']
y_train = train.drop(['ID','Tweet'],axis=1)
y_test = test.drop(['ID','Tweet'],axis=1)

In [ ]:
# Making Sure all the labels are an int datatype
y_train = y_train.apply(lambda x: list(map(float, x)))

In [ ]:
for column in y_test:
    y_test[column] = y_test[column].apply(lambda x: 0 if x == "NONE" else 1)
y_test = y_test.apply(lambda x: list(map(float, x)))

In [ ]:
DS = Dataset.from_dict({
    'texts': X_train,
    'labels': y_train.values
})

testset = Dataset.from_dict({
    'texts': X_test,
    'labels': y_test.values
})

In [ ]:
# Splitting the given train set into Trainset and Evaluation Set
train_eval_split = DS.train_test_split(test_size=0.2)
trainset = train_eval_split['train']
valset = train_eval_split['test']

In [ ]:
trainset.features

{'texts': Value(dtype='string', id=None),
 'labels': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None)}

In [ ]:
model_type = 't5-base'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_type)

def tokenize_fn(examples):
    tokenized_inputs = tokenizer(examples['texts'], padding="max_length", truncation=True,max_length=128)
    return tokenized_inputs


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
tokenized_trainsets = trainset.map(tokenize_fn, batched=True)
tokenized_valsets = valset.map(tokenize_fn, batched=True)
tokenized_testsets = testset.map(tokenize_fn, batched=True)

Map:   0%|          | 0/6179 [00:00<?, ? examples/s]

Map:   0%|          | 0/1545 [00:00<?, ? examples/s]

Map:   0%|          | 0/3259 [00:00<?, ? examples/s]

In [ ]:
tokenized_trainsets.set_format("torch")
tokenized_valsets.set_format("torch")
tokenized_testsets.set_format("torch")

In [ ]:
config = T5Config.from_pretrained(model_type, num_labels=11)

In [ ]:
config.id2label = {0:'anger', 1: 'anticipation',2: 'disgust',3: 'fear',4: 'joy',5: 'love',6: 'optimism',
  7:'pessimism', 8: 'sadness', 9: 'surprise', 10: 'trust'}

In [ ]:
config.label2id = {v: k for k, v in config.id2label .items()}

In [ ]:
config.problem_type = "multi_label_classification"

In [ ]:
model = T5ForSequenceClassification.from_pretrained(model_type, config=config).to('cuda')

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at t5-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    actual_logits = logits[0]

    # Applying sigmoid using numpy to convert logits to probabilities as the logits are
    # in the form of Numpy Array
    predictions = torch.sigmoid(torch.tensor(actual_logits)) > 0.5

    return {
        "f1": f1_score(labels, predictions, average="macro"),
        "accuracy": accuracy_score(labels, predictions),
    }


In [ ]:
training_args = TrainingArguments(
    output_dir=model_folder,
    evaluation_strategy='steps',  # Evaluate model at specified step intervals
    eval_steps=50,  # Perform evaluation every 10 training steps
    save_strategy="steps",  # Save model checkpoint at specified step intervals
    save_steps=100,
    save_total_limit=1,# Save a model checkpoint every 10 training steps
    learning_rate=3e-5,
    lr_scheduler_type='linear',
    num_train_epochs=10,
    optim='adamw_torch',
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    warmup_steps = 0,
    greater_is_better=True,
    weight_decay=0.01,
    disable_tqdm=False,
    logging_first_step=True,
    report_to='wandb',  # Log metrics and results to Weights & Biases platform
    run_name='Tweet_Emotion_FLANT5',
    fp16=True
)

In [ ]:
# early_stopping_callback = EarlyStoppingCallback(
#     early_stopping_patience=5,          # Number of evaluations to wait before stopping if no improvement
#     early_stopping_threshold=0.1      # Minimum improvement to qualify as an improvement
# )

In [ ]:
trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_trainsets,
        eval_dataset=tokenized_valsets,
        compute_metrics=compute_metrics,
)

In [ ]:
# specify the project name where the experiment will be logged
%env WANDB_PROJECT = nlp_course_spring_2024-HW6-Google-Flan-Fine-Tune

env: WANDB_PROJECT=nlp_course_spring_2024-HW6-Google-Flan-Fine-Tune


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,F1,Accuracy
50,0.297300,0.298511,0.511336,0.282848
100,0.297300,0.297227,0.527626,0.287379
150,0.297300,0.296579,0.525087,0.280906
200,0.297300,0.297414,0.533058,0.280259
250,0.297300,0.296634,0.530073,0.280906
300,0.297300,0.298292,0.532607,0.283495
350,0.297300,0.297938,0.537892,0.294498
400,0.297300,0.296730,0.537168,0.288026
450,0.297300,0.299848,0.539783,0.271845
500,0.247200,0.299014,0.545793,0.275728


Could not locate the best model at /content/drive/MyDrive/NLP/Assignments/HW6/Flan_T5/checkpoint-900/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/NLP/Assignments/HW6/Flan_T5/checkpoint-900'

In [ ]:
trainer.evaluate()

Step,Training Loss,Validation Loss,F1,Accuracy
50,0.297300,0.298511,0.511336,0.282848
100,0.297300,0.297227,0.527626,0.287379
150,0.297300,0.296579,0.525087,0.280906
200,0.297300,0.297414,0.533058,0.280259
250,0.297300,0.296634,0.530073,0.280906
300,0.297300,0.298292,0.532607,0.283495
350,0.297300,0.297938,0.537892,0.294498
400,0.297300,0.296730,0.537168,0.288026
450,0.297300,0.299848,0.539783,0.271845
500,0.247200,0.299014,0.545793,0.275728


{'eval_loss': 0.30092576146125793,
 'eval_f1': 0.5583561719139684,
 'eval_accuracy': 0.26990291262135924}

In [ ]:
eval_results = trainer.evaluate(tokenized_valsets)

Step,Training Loss,Validation Loss,F1,Accuracy
50,0.297300,0.298511,0.511336,0.282848
100,0.297300,0.297227,0.527626,0.287379
150,0.297300,0.296579,0.525087,0.280906
200,0.297300,0.297414,0.533058,0.280259
250,0.297300,0.296634,0.530073,0.280906
300,0.297300,0.298292,0.532607,0.283495
350,0.297300,0.297938,0.537892,0.294498
400,0.297300,0.296730,0.537168,0.288026
450,0.297300,0.299848,0.539783,0.271845
500,0.247200,0.299014,0.545793,0.275728


In [ ]:
eval_results

{'eval_loss': 0.30092576146125793,
 'eval_f1': 0.5583561719139684,
 'eval_accuracy': 0.26990291262135924}

In [ ]:
#Log results to Wandb
wandb.log({"eval_accuracy": eval_results["eval_accuracy"], "eval_loss": eval_results["eval_loss"], "eval_f1": eval_results["eval_f1"]})

In [ ]:
# After training, let us check the best checkpoint
# We need this for Inference
best_model_checkpoint_step = trainer.state.best_model_checkpoint.split('-')[-1]
print(f"The best model was saved at step {best_model_checkpoint_step}.")

The best model was saved at step 900.


In [ ]:
print(trainer.state.best_model_checkpoint)

/content/drive/MyDrive/NLP/Assignments/HW6/Flan_T5/checkpoint-900


In [ ]:
model_folder

'/content/drive/MyDrive/NLP/Assignments/HW6/Flan_T5'

In [ ]:
# Loading the saved checkpoint model
model_checkpoint = model_folder+'/checkpoint-'+best_model_checkpoint_step

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/NLP/Assignments/HW6/Flan_T5/checkpoint-1100')
model.to('cuda')
model.eval()

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=8,
    do_train=False,
    do_eval=True,
    report_to=[]
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_testsets,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

In [ ]:
trainer.evaluate(tokenized_valsets)

{'eval_loss': 0.3009267747402191,
 'eval_f1': 0.5584708952815121,
 'eval_accuracy': 0.26990291262135924,
 'eval_runtime': 89.4077,
 'eval_samples_per_second': 17.28,
 'eval_steps_per_second': 2.17}

In [ ]:
test_results = trainer.evaluate()

In [ ]:
test_results

{'eval_loss': 0.6017910838127136,
 'eval_f1': 0.0,
 'eval_accuracy': 0.023013194231359314,
 'eval_runtime': 131.1797,
 'eval_samples_per_second': 24.844,
 'eval_steps_per_second': 3.11}

In [ ]:
wandb.log({"test_accuracy": test_results["eval_accuracy"], "test_loss": test_results["eval_loss"], "test_f1": test_results["eval_f1"]})

In [ ]:
wandb.finish()

eval/accuracy,▁▁▄▆▆▅▅▅▅▅▆▇▇▇▇█▇▇█▇█████▇█████▇█▇▇▇▇▇▇▇
eval/f1,▁▁▄▅▅▅▅▅▅▅▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████████
eval/loss,█▇▅▄▃▃▃▃▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▂▁▂▂▁▁▁▂▃▁▃▂▇▅▃▂▂▃▃▃▂▄▃▂▂▃█▂▂▂▂▂▂▂▂▂▂▂▂▂
eval/samples_per_second,▆█▆▆▆▆▇▆▄▇▄▄▁▂▄▄▄▄▄▃▅▃▄▄▅▄▁▄▄▅▅▅▅▆▅▅▅▅▅▅
eval/steps_per_second,▂▃▂▂▂▂▂▂▁▂▆▇▂▃▆█▇▆▆▅█▄▆▇█▆▂▇▇▂▂▂▂▂▂▁▂▂▂▂
eval_accuracy,▁
eval_f1,▁
eval_loss,▁
test_accuracy,▁
test_f1,▁


## Model Inference

In [ ]:
#pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, config=config)

In [ ]:
#pipe("I am sad that, RCB has never won an IPL")

In [ ]:
#test['Tweet'][1]

In [ ]:
#pipe(test['Tweet'][1])

In [ ]:
#pipe("Food was awesome at that restaraunt")

In [ ]:
#pipe("Very sad that Dallas Cowboys have lost the game. Pain bro!")

In [ ]:
test_results = trainer.predict(tokenized_testsets)

In [ ]:
logits = test_results.predictions
probs = torch.sigmoid(torch.tensor(logits[0]))
threshold = 0.5
predictions = (probs > threshold).int().numpy()

In [ ]:
labels = [x for x in y_train.columns if x not in ['Tweet']]

In [ ]:
y_pred = pd.DataFrame(predictions,columns=labels)
output_submission = pd.concat([test['ID'],y_pred],axis=  1)
output_submission

,ID,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,2018-01559,1,0,1,1,0,0,0,0,0,0,0
1,2018-03739,1,0,1,0,0,0,0,0,1,0,0
2,2018-00385,1,0,1,0,0,0,0,0,1,0,0
3,2018-03001,0,1,0,0,1,0,1,0,0,0,0
4,2018-01988,0,0,0,1,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3254,2018-03848,1,0,1,0,0,0,0,0,1,0,0
3255,2018-00416,0,1,0,1,0,0,0,0,0,0,0
3256,2018-03717,1,0,1,0,0,0,0,0,0,0,0
3257,2018-03504,0,0,0,0,1,1,1,0,0,0,0


In [ ]:
output_submission.to_csv('/content/drive/MyDrive/NLP/Assignments/HW6/Flan_T5/output_submission.csv', index=False)

In [ ]:
test['Tweet'][3258]

"I'm the wholesome drunk that sends people memes and compliments them at 2am on snap"